In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
playa_att_df = pd.read_csv('../data/playa_nogeometry_whucs.csv')

In [ ]:
shp = '../data/shapefiles/playa_v5_summary_atts/playa_v5_summary_atts.shp'
playa_gdf = gpd.read_file(shp)
playa_gdf = playa_gdf[['id','geometry']].set_index('id')
playa_gdf['geometry'] = playa_gdf['geometry'].centroid
playa_gdf = playa_gdf.to_crs('epsg:4326')

In [ ]:
states_df = gpd.read_file('../data/other_data/us_states/cb_2019_us_state_500k.shp')
states_df = states_df[['NAME','geometry']].to_crs('epsg:4326')

In [ ]:
minx, miny, maxx, maxy = playa_gdf.geometry.total_bounds

# Plotting all playas, colored by whether all 0 or not

In [ ]:
max_inun_df = pd.read_csv('../data/max_inundation.csv').set_index('id')

In [ ]:
max_inun_df.rename(columns={'true':'max_inun'}, inplace=True)

In [ ]:
playa_gdf = playa_gdf.join(max_inun_df)

In [ ]:
ax = states_df.plot(color='white', edgecolor='black', figsize=[15,12])

playa_gdf.loc[playa_gdf['max_inun']==0].plot(ax=ax,  
                                            markersize=0.3, legend=False, color='#CD853F')
playa_gdf.loc[playa_gdf['max_inun']>0].plot(ax=ax, 
                                            markersize=0.3, legend=False, color='#20009F')
yellow_patch = mpatches.Patch(color='#CD853F', label='Never Inundated')
blue_patch = mpatches.Patch(color='#20009F', label='Inundated at least once')
plt.legend(handles=[yellow_patch, blue_patch],prop={'size': 14}, loc='upper right')
ax.tick_params(axis='both', which='major', labelsize=14)
plt.xlabel('Longitude',size=15)
plt.ylabel('Latitude',size=15)
ax.set_xlim(minx-0.5, maxx+0.5)
ax.set_ylim(miny-0.5, maxy+0.5)
plt.show()

# Per playa test loss

In [ ]:
test_loss_df = pd.read_csv('../data/per_playa_test_loss.csv', names=['id','test_loss']).iloc[1:].set_index('id')['test_loss']

In [ ]:
playa_gdf = playa_gdf.join(test_loss_df)

In [ ]:
playa_gdf = playa_gdf.assign(log_test_loss=np.log(playa_gdf['test_loss']))

In [ ]:
ax = states_df.plot(color='white', edgecolor='black', figsize=[15,9])
playa_gdf.plot(ax=ax, column='log_test_loss',  markersize=0.25, legend=True)
ax.set_xlim(minx-0.5, maxx+0.5)
ax.set_ylim(miny-0.5, maxy+0.5)
ax.tick_params(axis='both', which='major', labelsize=14)
ax.figure.axes[1].tick_params(labelsize=15)
plt.xlabel('Longitude',size=15)
plt.ylabel('Latitude',size=15)
plt.title('Log of Test Loss Per Playa', size=18)
plt.show()

## All loss

In [ ]:
full_loss_df = pd.read_csv('../data/per_playa_all_loss.csv', names=['id','full_loss']).iloc[1:].set_index('id')['full_loss']

In [ ]:
playa_gdf = playa_gdf.join(full_loss_df)

In [ ]:
ax = states_df.plot(color='white', edgecolor='black', figsize=[15,10])
playa_gdf.plot(ax=ax, column='full_loss',  markersize=0.25, vmin=0, vmax=0.5, legend=True)
ax.set_xlim(minx-0.5, maxx+0.5)
ax.set_ylim(miny-0.5, maxy+0.5)
plt.show()

## Test f1

In [ ]:
test_f1_df = pd.read_csv('../data/per_playa_test_f1.csv', names=['id','test_f1']).iloc[1:].set_index('id')['test_f1']

In [ ]:
playa_gdf = playa_gdf.join(test_f1_df)

In [ ]:
ax = states_df.plot(color='white', edgecolor='black', figsize=[15,10])
playa_gdf.plot(ax=ax,column='test_f1',  markersize=0.25, legend=True)
ax.set_xlim(minx-0.5, maxx+0.5)
ax.set_ylim(miny-0.5, maxy+0.5)
plt.show()

## Test f1 ignoring perfect 1s (which should all be playas with no inundation)

In [ ]:
ax = states_df.plot(color='white', edgecolor='black', figsize=[15,9])
playa_gdf.loc[playa_gdf['test_f1']<1].plot(ax=ax,column='test_f1',  markersize=0.25, vmin=0, vmax=1,
                                           legend=True)
ax.tick_params(axis='both', which='major', labelsize=14)
ax.figure.axes[1].tick_params(labelsize=15)
ax.set_xlim(minx-0.5, maxx+0.5)
ax.set_ylim(miny-0.5, maxy+0.5)
plt.xlabel('Longitude',size=15)
plt.ylabel('Latitude',size=15)
plt.title('Test F1 Per Playa\n(Excluding Non-Inundated Playas)', size=18)
plt.show()